### Import modules

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from ETGEMs_function_ETG import *
import pandas as pd
from ppmcot import *

In [2]:
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy
import math

### Data initialization

In [3]:
#Get Reaction G0 from local file_original
reaction_g0_file_original1 = './reaction_g0_ETC_butanoic_acid.txt'
reaction_g0_file_original0 = './reaction_g0_ETC_butanol.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original1 = './metabolites_lnC_ETC_butanoic_acid.txt'
metabolites_lnC_file_original0 = './metabolites_lnC_ETC_butanol.txt'
#Get Model from local file
model_file_original1 = './iML1515_butanoic_acid.xml'
model_file_original0 = './iML1515_butanol.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_ETC_butanoic_acid.csv'
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_ETC_butanol.csv'

In [4]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)

### Strain reconstruction

In [5]:
#gene deletion
model0['ub_list']['EX_but_e'] = 0
model0['lb_list']['PPC'] = 0
model0['lb_list']['LDH_D'] = 0
model0['lb_list']['ACALD'] = 0
model0['lb_list']['PTAr'] = 0
model0['ub_list']['PPC'] = 0
model0['ub_list']['LDH_D'] = 0
model0['ub_list']['ACALD'] = 0
model0['ub_list']['PTAr'] = 0
model0['lb_list']['ACACT1r'] = 0
model0['lb_list']['HACD1'] = 0
model0['lb_list']['ECOAH1'] = 0
model0['ub_list']['ACACT1r'] = 0
model0['ub_list']['HACD1'] = 0
model0['ub_list']['ECOAH1'] = 0

In [6]:
model1['ub_list']['BUTCT'] = 0
model1['ub_list']['EX_ac_e'] = 0
model1['lb_list']['PPC'] = 0
model1['lb_list']['LDH_D'] = 0
model1['lb_list']['ACALD'] = 0
model1['ub_list']['PPC'] = 0
model1['ub_list']['LDH_D'] = 0
model1['ub_list']['ACALD'] = 0
model1['ub_list']['PTAr'] = 0

### Predict the productions (g/L) under different inoculum ratios

In [7]:
#If you want to simulate the production with the unit of g/L, the real molar masses of exchange metabolites and carbon sources should be input, and the concentrations of exchange metabolites and carbon sources should be input with the unit of g/L
#If you want to simulate the production with the unit of mM, the molar masses of exchange metabolites and carbon sources should be input with the same value, and the concentrations of exchange metabolites and carbon sources should be input with the unit of mM

#model_list: the list of models calculated by the function of Get_Concretemodel_Need_Data
#target_list: the double list consist of the list of target reactions of each model, where the first reaction of each target reaction list is biomass biosynthesis reaction and the second is the reaction whose flux need to be maximized or minimized
#growth_list: the list of minimize growth rates in the function of MDF_Calculation
#growth_ratio: the list of ratios of lower bounds of growth rates to maximum growth rates
#parameter_list: the list of the total number of enzyme ratios, the upper bounds for substrate input reaction fluxes and the maximum values minus the minimum values of reaction thermodynamic driving forces
#substrate_environment: the list of the carbon source uptake reactions and the initial carbon source concentrations in environment
#metabolites_monitored: the list of metabolites delivered by the strains and the product
#metabolites_mass: the dictionary whose keys are the metabolites in metabolites_monitored and values are the molar masses of the metabolites
#concentrations_initial: the dictionary whose keys are the metabolites in metabolites_monitored and values are the initial concentrations of the metabolites in environment
#reactions_monitored: the list of dictionaries whose keys are the reactions of the metabolite uptake or release in metabolites_monitored and values are the ratios of lower bounds of the reaction fluxes to the max reaction fluxes
#substrate_mass: the list of the molar masses of the carbon sources
#t: the number of the time points for mono-cultivation
#culture_condition: the list of the inoculum ratio of the straints and time points for co-cultivation
#map_coefficient: the list of mapping constants of the instantaneous fluxes corresponding to the mean fluxes for the mono strains
#constraints: the constraints used in the simulation, where 'kinetics and thermodynamics' consist of FBA, kinetic and thermodynamic constraints, 'kinetics' consist of FBA and kinetic constraints, 'thermodynamics' consist of FBA and thermodynamic constraints and 'FBA' consist of FBA constraint

model_list=[model0, model1]
target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_btoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_but_e']]
growth_list=[0.2, 0.2]
growth_ratio=[0.34, 0.3]
parameter_list=[[0.19, 8, 1249], [0.19, 12, 1249]]
substrate_environment=[['EX_glc__D_e_reverse',20],['EX_glc__D_e_reverse',20]]
metabolites_monitored=['ac_e', 'but_e', 'btoh_e']
metabolites_mass={'ac_e': 60, 'but_e': 87, 'btoh_e': 74}
concentrations_initial={'ac_e': 0, 'but_e': 0, 'btoh_e': 0}
reactions_monitored=[{'EX_but_e_reverse': 0, 'EX_ac_e': 0, 'EX_btoh_e': 0}, {'EX_ac_e_reverse': 4/8, 'EX_but_e': 0}]
substrate_mass=[180,180]
t=60
culture_condition=[[1/4, 3/4], 60]
map_coefficient=[10/(10+10-7.8)*2, 12/(12+0.7)*2]
constraints='thermodynamics'

ppmcot(model_list, target_list, growth_list,growth_ratio, parameter_list, substrate_environment,  metabolites_monitored, metabolites_mass, concentrations_initial, reactions_monitored, substrate_mass, t, culture_condition, map_coefficient, constraints)

After 1 time point, the concentration of ac_e is 0.0018070523446051233 g/L or mM
After 1 time point, the concentration of but_e is 0.18320396843093245 g/L or mM
After 1 time point, the concentration of btoh_e is 0 g/L or mM
After 2 time point, the concentration of ac_e is 0.006493147175929717 g/L or mM
After 2 time point, the concentration of but_e is 0.3608585325563838 g/L or mM
After 2 time point, the concentration of btoh_e is 0.004697941514481741 g/L or mM
After 3 time point, the concentration of ac_e is 0.014812037680293521 g/L or mM
After 3 time point, the concentration of but_e is 0.5331799123866762 g/L or mM
After 3 time point, the concentration of btoh_e is 0.01395151989201842 g/L or mM
After 4 time point, the concentration of ac_e is 0.02653901620352062 g/L or mM
After 4 time point, the concentration of but_e is 0.700390107540434 g/L or mM
After 4 time point, the concentration of btoh_e is 0.027623975060657092 g/L or mM
After 5 time point, the concentration of ac_e is 0.04145

After 36 time point, the concentration of ac_e is 0.4078745070893739 g/L or mM
After 36 time point, the concentration of but_e is 4.190662787709797 g/L or mM
After 36 time point, the concentration of btoh_e is 2.180536061049842 g/L or mM
After 37 time point, the concentration of ac_e is 0.3954536481593578 g/L or mM
After 37 time point, the concentration of but_e is 4.254512116006878 g/L or mM
After 37 time point, the concentration of btoh_e is 2.2812284662754245 g/L or mM
After 38 time point, the concentration of ac_e is 0.38341103727838083 g/L or mM
After 38 time point, the concentration of but_e is 4.316226406515739 g/L or mM
After 38 time point, the concentration of btoh_e is 2.3827592170506278 g/L or mM
After 39 time point, the concentration of ac_e is 0.3717351557916215 g/L or mM
After 39 time point, the concentration of but_e is 4.375876443536838 g/L or mM
After 39 time point, the concentration of btoh_e is 2.485100280199913 g/L or mM
After 40 time point, the concentration of ac_

{'ac_e': 0.19417045481884532,
 'but_e': 5.255225699572441,
 'btoh_e': 4.780491782731829}

In [9]:
ppmcot(model_list=[model0, model1], target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_btoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_but_e']], growth_list=[0.2, 0.2],growth_ratio=[0.34, 0.3], parameter_list=[[0.19, 8, 1249], [0.19, 12, 1249]], substrate_environment=[['EX_glc__D_e_reverse',20],['EX_glc__D_e_reverse',20]],  metabolites_monitored=['ac_e', 'but_e', 'btoh_e'], metabolites_mass={'ac_e': 60, 'but_e': 87, 'btoh_e': 74}, concentrations_initial={'ac_e': 0, 'but_e': 0, 'btoh_e': 0}, reactions_monitored=[{'EX_but_e_reverse': 0, 'EX_ac_e': 0, 'EX_btoh_e': 0}, {'EX_ac_e_reverse': 4/8, 'EX_but_e': 0}], substrate_mass=[180,180], t=60, culture_condition=[[1/3, 2/3], 60], map_coefficient = [10/(10+10-7.8)*2, 12/(12+0.7)*2], constraints='thermodynamics')

After 1 time point, the concentration of ac_e is 0.002381884922533006 g/L or mM
After 1 time point, the concentration of but_e is 0.1609880575057743 g/L or mM
After 1 time point, the concentration of btoh_e is 0 g/L or mM
After 2 time point, the concentration of ac_e is 0.0068097513317169085 g/L or mM
After 2 time point, the concentration of but_e is 0.317168219408478 g/L or mM
After 2 time point, the concentration of btoh_e is 0.004081104309146091 g/L or mM
After 3 time point, the concentration of ac_e is 0.01438553669844988 g/L or mM
After 3 time point, the concentration of but_e is 0.4687184043053404 g/L or mM
After 3 time point, the concentration of btoh_e is 0.012121431070061993 g/L or mM
After 4 time point, the concentration of ac_e is 0.02491720133427679 g/L or mM
After 4 time point, the concentration of but_e is 0.6158290436094396 g/L or mM
After 4 time point, the concentration of btoh_e is 0.02400360872486661 g/L or mM
After 5 time point, the concentration of ac_e is 0.0382224

After 36 time point, the concentration of ac_e is 0.9826437366774552 g/L or mM
After 36 time point, the concentration of but_e is 3.7640721646562105 g/L or mM
After 36 time point, the concentration of btoh_e is 1.924968400496023 g/L or mM
After 37 time point, the concentration of ac_e is 0.9936113516827099 g/L or mM
After 37 time point, the concentration of but_e is 3.828622199434745 g/L or mM
After 37 time point, the concentration of btoh_e is 2.0203889637918797 g/L or mM
After 38 time point, the concentration of ac_e is 1.0022878506612805 g/L or mM
After 38 time point, the concentration of but_e is 3.8914203913134244 g/L or mM
After 38 time point, the concentration of btoh_e is 2.117445893344785 g/L or mM
After 39 time point, the concentration of ac_e is 1.008803088059405 g/L or mM
After 39 time point, the concentration of but_e is 3.952478963231408 g/L or mM
After 39 time point, the concentration of btoh_e is 2.2160947793164025 g/L or mM
After 40 time point, the concentration of ac_

{'ac_e': 0.8201491964354429,
 'but_e': 4.8737151916252115,
 'btoh_e': 4.582510209245359}

In [10]:
ppmcot(model_list=[model0, model1], target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_btoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_but_e']], growth_list=[0.2, 0.2],growth_ratio=[0.34, 0.3], parameter_list=[[0.19, 8, 1249], [0.19, 12, 1249]], substrate_environment=[['EX_glc__D_e_reverse',20],['EX_glc__D_e_reverse',20]],  metabolites_monitored=['ac_e', 'but_e', 'btoh_e'], metabolites_mass={'ac_e': 60, 'but_e': 87, 'btoh_e': 74}, concentrations_initial={'ac_e': 0, 'but_e': 0, 'btoh_e': 0}, reactions_monitored=[{'EX_but_e_reverse': 0, 'EX_ac_e': 0, 'EX_btoh_e': 0}, {'EX_ac_e_reverse': 4/8, 'EX_but_e': 0}], substrate_mass=[180,180], t=60, culture_condition=[[1/2, 1/2], 60], map_coefficient = [10/(10+10-7.8)*2, 12/(12+0.7)*2], constraints='thermodynamics')

After 1 time point, the concentration of ac_e is 0.0034902727729780344 g/L or mM
After 1 time point, the concentration of but_e is 0.11795117148008227 g/L or mM
After 1 time point, the concentration of btoh_e is 0 g/L or mM
After 2 time point, the concentration of ac_e is 0.007602749785495136 g/L or mM
After 2 time point, the concentration of but_e is 0.23248877991360228 g/L or mM
After 2 time point, the concentration of btoh_e is 0 g/L or mM
After 3 time point, the concentration of ac_e is 0.013787989078820211 g/L or mM
After 3 time point, the concentration of but_e is 0.34372307243187505 g/L or mM
After 3 time point, the concentration of btoh_e is 0.005757492332593573 g/L or mM
After 4 time point, the concentration of ac_e is 0.022074269995110845 g/L or mM
After 4 time point, the concentration of but_e is 0.4517840316632645 g/L or mM
After 4 time point, the concentration of btoh_e is 0.014269657757511243 g/L or mM
After 5 time point, the concentration of ac_e is 0.03233467381239557 g

After 36 time point, the concentration of ac_e is 0.8616645625379027 g/L or mM
After 36 time point, the concentration of but_e is 2.7853458642940527 g/L or mM
After 36 time point, the concentration of btoh_e is 1.3841647536758157 g/L or mM
After 37 time point, the concentration of ac_e is 0.8950638283469605 g/L or mM
After 37 time point, the concentration of but_e is 2.8348268634524105 g/L or mM
After 37 time point, the concentration of btoh_e is 1.4531427381757704 g/L or mM
After 38 time point, the concentration of ac_e is 0.9284946143653368 g/L or mM
After 38 time point, the concentration of but_e is 2.8833638716662784 g/L or mM
After 38 time point, the concentration of btoh_e is 1.5233460999750914 g/L or mM
After 39 time point, the concentration of ac_e is 0.9619366531154212 g/L or mM
After 39 time point, the concentration of but_e is 2.930985247415558 g/L or mM
After 39 time point, the concentration of btoh_e is 1.5947514615128173 g/L or mM
After 40 time point, the concentration of

{'ac_e': 1.64127657235869,
 'but_e': 3.7592962860651755,
 'btoh_e': 3.336951518808464}

In [11]:
ppmcot(model_list=[model0, model1], target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_btoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_but_e']], growth_list=[0.2, 0.2],growth_ratio=[0.34, 0.3], parameter_list=[[0.19, 8, 1249], [0.19, 12, 1249]], substrate_environment=[['EX_glc__D_e_reverse',20],['EX_glc__D_e_reverse',20]],  metabolites_monitored=['ac_e', 'but_e', 'btoh_e'], metabolites_mass={'ac_e': 60, 'but_e': 87, 'btoh_e': 74}, concentrations_initial={'ac_e': 0, 'but_e': 0, 'btoh_e': 0}, reactions_monitored=[{'EX_but_e_reverse': 0, 'EX_ac_e': 0, 'EX_btoh_e': 0}, {'EX_ac_e_reverse': 4/8, 'EX_but_e': 0}], substrate_mass=[180,180], t=60, culture_condition=[[2/3, 1/3], 60], map_coefficient = [10/(10+10-7.8)*2, 12/(12+0.7)*2], constraints='thermodynamics')

After 1 time point, the concentration of ac_e is 0.004543624216208745 g/L or mM
After 1 time point, the concentration of but_e is 0.07677419988722253 g/L or mM
After 1 time point, the concentration of btoh_e is 0 g/L or mM
After 2 time point, the concentration of ac_e is 0.009847166125949732 g/L or mM
After 2 time point, the concentration of but_e is 0.15141420881074483 g/L or mM
After 2 time point, the concentration of btoh_e is 0 g/L or mM
After 3 time point, the concentration of ac_e is 0.015060905701068274 g/L or mM
After 3 time point, the concentration of but_e is 0.2239930606304229 g/L or mM
After 3 time point, the concentration of btoh_e is 0 g/L or mM
After 4 time point, the concentration of ac_e is 0.021248310596605185 g/L or mM
After 4 time point, the concentration of but_e is 0.2945685494165037 g/L or mM
After 4 time point, the concentration of btoh_e is 0.005415894968747251 g/L or mM
After 5 time point, the concentration of ac_e is 0.02867226592642124 g/L or mM
After 5 time

After 36 time point, the concentration of ac_e is 0.5825675003885014 g/L or mM
After 36 time point, the concentration of but_e is 1.8342977216481924 g/L or mM
After 36 time point, the concentration of btoh_e is 0.8832648125909336 g/L or mM
After 37 time point, the concentration of ac_e is 0.6047112591106085 g/L or mM
After 37 time point, the concentration of but_e is 1.8672752117020175 g/L or mM
After 37 time point, the concentration of btoh_e is 0.9276160249628537 g/L or mM
After 38 time point, the concentration of ac_e is 0.6268788524691966 g/L or mM
After 38 time point, the concentration of but_e is 1.8996367131899055 g/L or mM
After 38 time point, the concentration of btoh_e is 0.9727645951723867 g/L or mM
After 39 time point, the concentration of ac_e is 0.649057274214725 g/L or mM
After 39 time point, the concentration of but_e is 1.9314002695531676 g/L or mM
After 39 time point, the concentration of btoh_e is 1.0186956293234182 g/L or mM
After 40 time point, the concentration of

{'ac_e': 1.1007702510354898,
 'but_e': 2.485857684904956,
 'btoh_e': 2.141614331952453}